In [151]:
from importlib import reload
from bowzer import model, data_processing, constants
reload(model)
reload(data_processing)
reload(constants)

#from bowzer import train
#reload(train)


<module 'bowzer.constants' from '/Volumes/KCQDrive/projects/bowzer_drive/bowzer/constants.py'>

In [152]:
from bowzer.constants import (
    RESIZE_N, 
    DIR
)

In [153]:

from datetime import datetime
import torch
from torch.nn import CrossEntropyLoss
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [156]:
data_module = data_processing.Dataset(RESIZE_N)
dataloader_train, dataloader_test = data_module.process()
num_classes = data_module.num_classes
net = model.Net(data_module.num_classes,  data_module.resize_n).to(device)

loss_fn = CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader_train, net, loss_fn, optimizer)
    test(dataloader_test, net, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 3.620772  [  256/ 3680]
Test Error: 
 Accuracy: 4.3%, Avg loss: 6.965129 

Epoch 2
-------------------------------
loss: 6.369938  [  256/ 3680]


KeyboardInterrupt: 

In [115]:

epochs = 3
best_loss = 1_000_000
counter = 0
# Define the loss function
criterion = CrossEntropyLoss()
# Define the optimizer
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)
for epoch in range(epochs):
    running_loss = 0.0
    # Iterate over training batches
    for images, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / len(dataloader_train)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

Epoch 1, Loss: 4.5159


KeyboardInterrupt: 

In [113]:
def train_epoch(epoch_index, net, criterion, optimizer, dataloader_train):
    running_loss, last_loss = 0, 0
    for i, data in enumerate(dataloader_train):
        images, labels = data
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000
            print(f"batch {i + 1} loss: {last_loss}")
            running_loss = 0
    return last_loss

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

data_module = Dataset(RESIZE_N)
dataloader_train, dataloader_test = data_module.process()
num_classes = data_module.num_classes
net = model.Net(data_module.num_classes,  data_module.resize_n)

epochs = 3
best_loss = 1_000_000
counter = 0
# Define the loss function
criterion = nn.CrossEntropyLoss()
# Define the optimizer
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)
for epoch in range(epochs):
    print(f"EPOCH {counter + 1}:")
    net.train(True)
    avg_loss = train_epoch(counter, net, criterion, optimizer, dataloader_train)
    running_loss = 0.0
    net.eval()
    with torch.no_grad():
        for i, data in enumerate(dataloader_test):
            images, labels = data
            outputs = net(images)
            loss = criterion(outputs, labels)
            running_loss += loss
    avg_test_loss = running_loss / (i + 1)
    print(f"LOSS Train {avg_loss} Test: {avg_test_loss}")

    if avg_test_loss < best_loss:
        best_loss = avg_test_loss
        torch.save(net.state_dict(), f'{DIR}/bowzer/runs/trainer_{timestamp}/model_{timestamp}_{counter}')
    counter += 1


EPOCH 1:


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn

def train_epoch(self, epoch_index):
    running_loss, last_loss = 0, 0
    for i, data in enumerate(self.dataloader_train):
        images, labels = data
        self.optimizer.zero_grad()
        outputs = self.net(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000
            print(f"batch {i + 1} loss: {last_loss}")
            running_loss = 0
    return last_loss

def train_eval(self, epochs:int = 5):
    best_loss = 1_000_000
    counter = 0
    #Define the model
    self.net = Net(num_classes=self.num_classes, resize_n=self.resize_n)
    # Define the loss function
    self.criterion = nn.CrossEntropyLoss()
    # Define the optimizer
    self.optimizer = torch.optim.Adam(self.net.parameters(), lr = 0.001)
    for epoch in range(epochs):
        print(f"EPOCH {counter + 1}:")
        self.net.train(True)
        avg_loss = self.train_epoch(counter)

        running_loss = 0.0
        self.net.eval()
        with torch.no_grad():
            for i, data in enumerate(self.dataloader_test):
                images, labels = data
                outputs = self.net(images)
                loss = self.criterion(outputs, labels)
                running_loss += loss
        avg_test_loss = running_loss / (i + 1)
        print(f"LOSS Train {avg_loss} Test: {avg_test_loss}")

        if avg_test_loss < best_loss:
            best_loss = avg_test_loss
            torch.save(self.net.state_dict(), f'{DIR}/bowzer/runs/trainer_{self.timestamp}/model_{self.timestamp}_{counter}')
        counter += 1


In [92]:
model = train.Model(resize_n=RESIZE_N)

AttributeError: cannot assign module before Module.__init__() call

In [89]:
model.train()

AttributeError: module 'bowzer.model' has no attribute 'train'

In [29]:
help(model.dataloader_train)

Help on DataLoader in module torch.utils.data.dataloader object:

class DataLoader(typing.Generic)
 |  DataLoader(dataset: torch.utils.data.dataset.Dataset[+T_co], batch_size: Optional[int] = 1, shuffle: Optional[bool] = None, sampler: Union[torch.utils.data.sampler.Sampler, Iterable, NoneType] = None, batch_sampler: Union[torch.utils.data.sampler.Sampler[List], Iterable[List], NoneType] = None, num_workers: int = 0, collate_fn: Optional[Callable[[List[~T]], Any]] = None, pin_memory: bool = False, drop_last: bool = False, timeout: float = 0, worker_init_fn: Optional[Callable[[int], NoneType]] = None, multiprocessing_context=None, generator=None, *, prefetch_factor: Optional[int] = None, persistent_workers: bool = False, pin_memory_device: str = '')
 |  
 |  Data loader combines a dataset and a sampler, and provides an iterable over the given dataset.
 |  
 |  The :class:`~torch.utils.data.DataLoader` supports both map-style and
 |  iterable-style datasets with single- or multi-process 